In [1]:
# # GPU 확인 
# from tensorflow.python.client import device_lib
# device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 2120214103554001710
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 6259015680
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 6992225141639690626
 physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 2080, pci bus id: 0000:01:00.0, compute capability: 7.5"
 xla_global_id: 416903419]

In [3]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split

In [5]:
import os
# csv파일 경로 리스팅 
data_lis = os.listdir('./')
data_lis

['.git',
 '.ipynb_checkpoints',
 '0411_RNN_LSTM.ipynb',
 '20220418.ipynb',
 'BERT.ipynb',
 'best_model',
 'chromedriver.exe',
 'data',
 'gitignore.txt',
 'investing_scrap.ipynb',
 'labeled_news_oil_0_100_oil_price.csv.csv',
 'LDA 모델 구축.ipynb',
 'LDA모델구축_최종.ipynb',
 'Reuter_Crawling_Code.ipynb',
 'RNN_CNBC뉴스 분류  - 복사본.ipynb',
 'RNN_CNBC뉴스 분류 .ipynb',
 'tokenizer.pickle',
 'Tokenizing.ipynb',
 '당일뉴스분류_0422.csv',
 '석유와가스_지수.csv',
 '카테고리35개_분류모델.ipynb',
 '카테고리분류.ipynb',
 '카테고리분류2.ipynb',
 '카테고리분류모델피클',
 '토크나이징']

In [1]:
pwd

'C:\\Users\\bitcamp\\Project4'

In [63]:
cd C:\Users\bitcamp\Project4\data\1차뉴스분류-20220425T013418Z-001\1차뉴스분류\commodities

C:\Users\bitcamp\Project4\data\1차뉴스분류-20220425T013418Z-001\1차뉴스분류\commodities


In [17]:
# csv파일 여러개 읽어와서 합치기 
df_cnbc = pd.DataFrame()

for files in data_lis:
    # advertisement 빼고 합치기 
    if "당일뉴스" in files:
        df = pd.read_csv(files,index_col=0)
        df_cnbc = pd.concat([df_cnbc,df])  

In [18]:
df = df_cnbc

In [19]:
df.head()

,date,news_oil,news_ship,news_sc,news_steel,news_vh,news_others
0,2022-04-22,"As a candidate, Joe Biden made climate change ...",NaN,NaN,NaN,NaN,NaN
1,2022-04-22,By David Gaffen\nNEW YORK (Reuters) -Oil price...,NaN,NaN,NaN,NaN,NaN
2,2022-04-22,By Barani Krishnan\nInvesting.com -- Oil price...,NaN,NaN,NaN,NaN,NaN
3,2022-04-22,NaN,NaN,NaN,NaN,NaN,(Bloomberg) -- The U.S. West Coast is importin...
4,2022-04-22,U.S. companies can help rebuild Ukraine with d...,NaN,NaN,NaN,NaN,NaN


In [14]:
df = df[['date','news_oil']]

In [15]:
df.head()

,date,news_oil
0,"<span>Apr 15, 2022 04:50PM ET</span>",NaN
1,"<span>Apr 15, 2022 04:30PM ET</span>",NaN
2,"<span>Apr 15, 2022 12:35AM ET</span>",NaN
3,"<span>Apr 14, 2022 02:00PM ET</span>",NaN
4,"<span>Apr 14, 2022 11:41AM ET</span>",By Timothy Aeppel\n(Reuters) - Through most of...


In [25]:
df = df.dropna().reset_index().drop('index',axis=1)

In [27]:
df.sort_values('date')

,date,news_others
0,2022-04-22,(Bloomberg) -- The U.S. West Coast is importin...
111,2022-04-22,"(Reuters) -The head of luxury house LVMH, Bern..."
112,2022-04-22,Union Pacific (NYSE:UNP) reported on Thursda...
113,2022-04-22,By Peter Nurse\nInvesting.com -- Stocks in foc...
114,2022-04-22,"Generali (BIT:GASI) boss Philippe Donnet, faci..."
...,...,...
60,2022-04-22,Gap Inc. (NYSE:GPS) shares were trading more t...
61,2022-04-22,Boston Beer Co. (NYSE:SAM) shares were trading...
62,2022-04-22,Mexico's Grupo Financiero Banorte on Thursday ...
54,2022-04-22,By Geoffrey Smith \nInvesting.com -- The euro ...


In [22]:
df = df[['date','news_others']]

In [9]:
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from bs4 import BeautifulSoup
import re

In [10]:
def preprocessing(docs):
    tokenizer = RegexpTokenizer('[\w]+')
    stop_words = stopwords.words('english')
    p_stemmer = PorterStemmer()  # 어근 복원 : runs, running, ran => run,  cars => car
#     p = re.compile("[0-9]+")
#     숫자제거해야됨
#     docs = [p.sub(i) for i in docs]
    docs = BeautifulSoup(docs, "html5lib").get_text()
    low = docs.lower()
    tokens = tokenizer.tokenize(low)
    stopped_tokens = [i for i in tokens if not i in stop_words]
    stopped_tokens = [re.sub('[^a-zA-Z0-9]',' ',i) for i in stopped_tokens]
    stemmer_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
    one_rmv = [w for w in stemmer_tokens if len(w)>1]
  
    return one_rmv

In [28]:
# import nltk
# nltk.download('stopwords')
# df['date']= df['date'].apply(preprocessing)
df['clean_others'] = df['news_others'].apply(preprocessing)

In [29]:
df

,date,news_others,clean_others
0,2022-04-22,(Bloomberg) -- The U.S. West Coast is importin...,"[bloomberg, west, coast, import, gasolin, near..."
1,2022-04-22,U.S. farmers are off to a slow start in their ...,"[farmer, slow, start, corn, plant, year, farme..."
2,2022-04-22,Exxon Mobil Corp (NYSE:XOM). estimates there...,"[exxon, mobil, corp, nyse, xom, estim, trillio..."
3,2022-04-22,from Apple (NASDAQ:AAPL) and Boeing (NYSE:BA) ...,"[appl, nasdaq, aapl, boe, nyse, ba, bp, nyse, ..."
4,2022-04-22,OPEC told the International Monetary Fund's st...,"[opec, told, intern, monetari, fund, steer, co..."
...,...,...,...
169,2022-04-22,U.S. President Joe Biden will unveil a new str...,"[presid, joe, biden, unveil, new, strategi, de..."
170,2022-04-22,their only one of the campaign,"[one, campaign]"
171,2022-04-22,By Kanishka Singh and David Shepardson\nWASHIN...,"[kanishka, singh, david, shepardson, washingto..."
172,2022-04-22,suddenly in my 50s,"[suddenli, 50]"


In [120]:
pwd

'C:\\Users\\bitcamp\\Project4\\data\\investing.com news-20220425T032341Z-001\\investing.com news\\commodities-news'

In [128]:
cd C:/Users/bitcamp/Project4/best_model/

C:\Users\bitcamp\Project4\best_model


In [22]:
df = df.sort_values('date')

In [23]:
df

,date,news_oil
7,"<span>Apr 01, 2022 03:31PM ET</span>","[howard, schneider, dan, burn, reuter, coronav..."
6,"<span>Apr 05, 2022 02:01PM ET</span>","[lucia, mutikani, washington, reuter, servic, ..."
230,"<span>Apr 06, 2021 05:00AM ET</span>","[emma, pinedo, aida, pelaez, fernandez, madrid..."
229,"<span>Apr 08, 2021 07:30AM ET</span>","[david, milliken, london, reuter, british, hou..."
228,"<span>Apr 09, 2021 06:55AM ET</span>","[leika, kihara, tokyo, reuter, number, japan, ..."
...,...,...
331,"<span>Sep 30, 2020 06:15AM ET</span>","[michael, nienab, berlin, reuter, german, reta..."
110,"<span>Sep 30, 2021 01:16PM ET</span>","[lucia, mutikani, washington, reuter, number, ..."
109,"<span>Sep 30, 2021 04:37PM ET</span>","[reuter, retail, neiman, marcu, group, said, t..."
112,"<span>Sep 30, 2021 05:31AM ET</span>","[hong, kong, reuter, hong, kong, retail, sale,..."


In [97]:
# 전처리 

In [30]:
preprocessed_sentences = df['clean_others']

In [31]:
import pickle
import re
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [32]:
x = tokenizer.texts_to_sequences(preprocessed_sentences)

In [131]:
print(x)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [34]:
max_len = max([len(i) for i in df['clean_others']]) 
max_len

604

In [35]:
x = pad_sequences(x, maxlen=max_len)

In [70]:
import tensorflow as tf
import numpy as np

##########모델 로드

model = tf.keras.models.load_model('./best_model/best_mode5l.h5')

In [71]:
y_predict = model.predict(x)

In [72]:
y_predict

array([[9.89150643e-01, 6.78305840e-03, 8.55556573e-04, 1.25571771e-03,
        1.95496972e-03],
       [1.41774900e-02, 2.25481801e-02, 7.06865907e-01, 2.30162993e-01,
        2.62455493e-02],
       [7.84067035e-01, 2.23629493e-02, 3.05412784e-02, 1.14449402e-02,
        1.51583776e-01],
       [7.92556167e-01, 1.93336770e-01, 4.75521665e-03, 4.00201464e-03,
        5.34978928e-03],
       [8.71373415e-01, 1.01980112e-01, 6.24768529e-03, 1.56076886e-02,
        4.79118060e-03],
       [1.28283473e-02, 5.63352881e-03, 4.27409541e-03, 6.30445080e-04,
        9.76633608e-01],
       [2.90972918e-01, 1.09106004e-01, 2.11802587e-01, 2.08881572e-01,
        1.79236963e-01],
       [9.85243559e-01, 9.81700141e-03, 9.83484671e-04, 2.63881776e-03,
        1.31723075e-03],
       [7.79812643e-03, 1.52407046e-02, 2.18298696e-02, 9.47781861e-01,
        7.34942826e-03],
       [9.77126002e-01, 1.06781842e-02, 3.51322792e-03, 3.56390071e-03,
        5.11871185e-03],
       [6.87134862e-02, 7.0675

In [68]:
from keras import models 
model = models.load_model(model_path, custom_objects= {'f1_score': f1_score})

AttributeError: 'function' object has no attribute 'evaluate'

In [60]:
predict_lis = []
for i in range(len(y_predict)):
    predict_lis.append(np.argmax(y_predict[i]))

In [41]:
import collections
# apple3개, banana2개, cherry는 4개
# list = ['apple', 'apple', 'apple', 'banana', 'banana',
# 'cherry', 'cherry', 'cherry', 'cherry']
# 빈도수를 세려면?
counts = collections.Counter(리스)
print(counts)
# 가장 흔하게 등장하는 단어의 첫번째 값을 most_common(1)로 추출
# most_common(1) => [('cherry',4)] _리스트
# most_common(1)[0] => ('cherry', 4) _ 튜플
# most_common(1)[0][0] => cherry _ 튜플에서 첫번째 추출
print('가장 많이 예측한 라벨은?: ', counts.most_common(1)[0][0])


Counter({3: 64, 1: 46, 0: 26, 2: 25, 4: 13})
가장 많이 예측한 라벨은?:  3


In [61]:
print(predict_lis[:20])

[0, 2, 0, 0, 0, 4, 0, 0, 3, 0, 1, 0, 0, 0, 0, 0, 1, 3, 1, 1]


In [ ]:
# 0 = oil 
# 1 = semi 
# 2 = ship 
# 3 = steel
# 4 = vehicle 


In [49]:
df['news_others'][0]

'(Bloomberg) -- The U.S. West Coast is importing gasoline at a near-record pace this month to help ease supply constraints amid eye-popping pump prices just ahead of the peak summer driving season.\nAround 120,000 barrels a day of gasoline and blending components are expected to reach the U.S. West Coast in April, including 95,000 barrels a day that have already arrived, marking the second-highest monthly level since oil analytics firm Vortexa began tracking the data in January 2016. The region imported 150,000 barrels a day in May 2019.\nThese imports underscore the need to replenish supply ahead of what’s shaping up to be a tight summer for gasoline. A heavy maintenance season that’s just now winding down left West Coast refinery rates at 84% last week. That has put additional pressure on drivers, with Californians paying more than anywhere else in the U.S. for gasoline.\n Chevron  Corp. (NYSE:CVX) is due to carry out major maintenance in June at its Richmond, California refinery, on

In [48]:
print(collections.Counter(df['clean_others'][0])) #ship

Counter({'gasolin': 7, 'west': 5, 'coast': 5, 'import': 4, 'price': 4, 'month': 3, 'suppli': 3, 'pump': 3, 'peak': 3, 'season': 3, '000': 3, 'barrel': 3, 'day': 3, 'refineri': 3, 'demand': 3, 'gallon': 3, 'bloomberg': 2, 'near': 2, 'record': 2, 'ahead': 2, 'summer': 2, 'reach': 2, 'april': 2, 'monthli': 2, 'level': 2, 'may': 2, 'mainten': 2, 'driver': 2, 'californian': 2, 'pay': 2, 'california': 2, 'meanwhil': 2, 'high': 2, 'fuel': 2, 'march': 2, 'year': 2, 'averag': 2, 'nation': 2, 'volum': 2, 'export': 2, 'pace': 1, 'help': 1, 'eas': 1, 'constraint': 1, 'amid': 1, 'eye': 1, 'pop': 1, 'drive': 1, 'around': 1, '120': 1, 'blend': 1, 'compon': 1, 'expect': 1, 'includ': 1, '95': 1, 'alreadi': 1, 'arriv': 1, 'mark': 1, 'second': 1, 'highest': 1, 'sinc': 1, 'oil': 1, 'analyt': 1, 'firm': 1, 'vortexa': 1, 'began': 1, 'track': 1, 'data': 1, 'januari': 1, '2016': 1, 'region': 1, '150': 1, '2019': 1, 'underscor': 1, 'need': 1, 'replenish': 1, 'shape': 1, 'tight': 1, 'heavi': 1, 'wind': 1, 'left

In [58]:
df['news_others'][17]

'By Lucy Craymer\nWELLINGTON (Reuters) -New Zealand\'s consumer prices rose at the fastest pace in three decades last quarter, underlining the need for the central bank to stay on its hawkish course to contain price pressures without tipping the economy into recession.\nThe New Zealand dollar slipped after the data showed inflation was not quite as hot as feared though, slightly softening expectations the central bank would again hike rates by 50 basis points in May.\nAnnual inflation rose 6.9% in the first quarter from 5.9% in the fourth quarter, the fastest since a 7.6% clip in the June quarter of 1990, Statistics New Zealand said in a statement on Thursday\nCPI rose 1.8% in the quarter ending March from a 1.4% rise in the fourth quarter. But the data was below economists’ expectations in a Reuters poll that forecast a 2.0% rise for the quarter, and an annual rise of 7.1%.\nThe Reserve Bank of New Zealand (RBNZ) raised interest rates by a hefty 50 basis points to 1.50% last Wednesday

In [62]:
print(collections.Counter(df['clean_others'][1])) #semi 

Counter({'farmer': 7, 'corn': 7, 'year': 6, 'crop': 4, 'plant': 3, 'forecast': 3, 'ukrain': 3, 'grain': 3, 'product': 3, 'said': 3, 'weather': 3, 'seed': 3, 'start': 2, 'key': 2, 'area': 2, 'next': 2, 'week': 2, 'temperatur': 2, 'southern': 2, 'midwest': 2, 'need': 2, 'left': 2, 'russia': 2, 'invas': 2, 'export': 2, '2022': 2, 'import': 2, 'produc': 2, 'agricultur': 2, 'north': 2, 'perfect': 2, '17': 2, 'world': 2, 'million': 2, 'tonn': 2, 'streit': 2, 'slow': 1, 'mani': 1, 'grow': 1, 'like': 1, 'remain': 1, 'sidelin': 1, 'much': 1, 'two': 1, 'latest': 1, 'show': 1, 'shower': 1, 'cool': 1, 'mississippi': 1, 'river': 1, 'delta': 1, 'big': 1, 'help': 1, 'fill': 1, 'gap': 1, 'suppli': 1, 'stock': 1, 'last': 1, 'strand': 1, 'countri': 1, 'threaten': 1, 'curtail': 1, 'unabl': 1, 'due': 1, 'fight': 1, 'incred': 1, 'maximum': 1, 'keep': 1, 'price': 1, 'stabl': 1, 'sterl': 1, 'smith': 1, 'director': 1, 'research': 1, 'agrisompo': 1, 'america': 1, 'go': 1, 'right': 1, 'chicago': 1, 'board': 1, 

In [63]:
df['news_others'][1]

'U.S. farmers are off to a slow start in their corn planting this year and farmers in many key growing areas will likely remain sidelined for much of the next two weeks as the latest forecasts show showers and cool temperatures in the southern Midwest and Mississippi River Delta.\nA big U.S. crop is needed this year to help fill the gap in supply left by Russia\'s invasion of Ukraine, which has left grain stocks from last year stranded in that key export country and threatens to curtail production in 2022 as farmers there are unable to plant due to the fighting.\n"It is incredibly important that we produce a maximum crop to keep prices stable," said Sterling Smith, director of agricultural research at AgriSompo North America. "We are going to need perfect weather and right now we do not have perfect weather." \nChicago Board of Trade corn futures have surged 17.5% since Russia\'s invasion, with the most-active contract nearing a 10-year high, as importers have scrambled to find alterna

In [152]:
print(collections.Counter(df['content'][2])) #semi 

Counter({'fire': 5, 'refineri': 5, 'pemex': 4, 'mexico': 3, 'salina': 3, 'cruz': 3, 'injuri': 3, 'reuter': 2, 'state': 2, 'oil': 2, 'said': 2, 'brought': 2, 'control': 2, 'without': 2, 'tank': 2, 'anoth': 2, 'citi': 1, 'mexican': 1, 'compani': 1, 'saturday': 1, 'southern': 1, 'oaxaca': 1, 'personnel': 1, 'start': 1, 'friday': 1, 'engulf': 1, 'gasolin': 1, 'storag': 1, 'send': 1, 'dens': 1, 'plume': 1, 'black': 1, 'smoke': 1, 'billow': 1, 'insid': 1, 'firefight': 1, 'conclud': 1, 'work': 1, 'suffoc': 1, 'cool': 1, 'august': 1, 'hit': 1, 'capac': 1, 'process': 1, '330': 1, '000': 1, 'barrel': 1, 'per': 1, 'day': 1, 'crude': 1, 'accord': 1, 'data': 1, 'eikon': 1, 'refinitiv': 1, 'novemb': 1, 'broke': 1, 'complex': 1, 'northern': 1, 'staff': 1, 'disrupt': 1, 'product': 1, 'http': 1, 'www': 1, 'com': 1, 'busi': 1, 'energi': 1, 'break': 1, 'cadereyta': 1, 'report': 1, 'sourc': 1, '2021': 1, '11': 1, '12': 1})


In [56]:
df['news_others'][5]

'Some U.S. oilfield firms whose Venezuelan operations were frozen by sanctions are joining an appeal to Washington for authorizations to restart oil drilling in the South American country, according to eight sources with knowledge of the talks.\nIf they are allowed to resume work, Venezuela could quickly ramp up production capacity beyond 1 million barrels per day (bpd), analysts said. \nThe new supply could also fill a void left by U.S. ban on Russian energy imports over its invasion of Ukraine that has contributed to crude prices above $100 per barrel. \nSchlumberger (NYSE:SLB), Halliburton (NYSE:HAL), Baker Hughes and Weatherford International (NASDAQ:WFRD) have been barred since 2019 from helping Venezuelan state-run PDVSA and its joint ventures produce oil. Any easing by the U.S. Treasury Department of a restricted license the four companies share with  Chevron  Corp (NYSE:CVX) could let them expand operations.\nSince the United States first imposed oil trading sanctions on Venezu

In [153]:
print(collections.Counter(df['content'][3])) #vehicle

Counter({'flock': 6, 'poultri': 6, 'said': 5, 'state': 5, 'bird': 4, 'outbreak': 4, 'vaccin': 4, 'flu': 3, 'pennsylvania': 3, 'utah': 3, 'countri': 3, 'test': 3, 'laboratori': 3, 'usda': 3, 'import': 3, 'case': 2, 'commerci': 2, 'chicken': 2, 'depart': 2, 'saturday': 2, 'option': 2, 'sinc': 2, 'sampl': 2, 'veterinari': 2, 'health': 2, 'work': 2, 'addit': 2, 'unit': 2, 'world': 2, 'export': 2, 'kanishka': 1, 'singh': 1, 'washington': 1, 'reuter': 1, 'new': 1, 'detect': 1, 'backyard': 1, 'agricultur': 1, 'spread': 1, '30': 1, 'earlier': 1, 'month': 1, 'look': 1, 'protect': 1, 'deadli': 1, 'face': 1, 'worst': 1, '2015': 1, 'current': 1, 'wipe': 1, 'well': 1, 'twenti': 1, 'million': 1, 'turkey': 1, 'februari': 1, 'nyse': 1, 'lh': 1, 'diagnost': 1, 'part': 1, 'nation': 1, 'anim': 1, 'network': 1, 'statement': 1, 'feder': 1, 'offici': 1, 'jointli': 1, 'surveil': 1, 'area': 1, 'around': 1, 'affect': 1, 'recent': 1, 'highli': 1, 'pathogen': 1, 'avian': 1, 'influenza': 1, 'hpai': 1, 'present': 

In [154]:
print(collections.Counter(df['content'][4])) #steel

Counter({'macron': 10, 'melenchon': 6, 'franc': 5, 'first': 5, 'voter': 5, 'runoff': 5, 'vote': 5, 'left': 5, 'le': 5, 'pen': 5, 'said': 5, 'marseil': 4, 'plan': 4, 'want': 4, 'saturday': 3, 'nation': 3, 'coal': 3, 'ga': 3, 'energi': 3, 'wing': 3, 'round': 3, 'seek': 3, 'support': 3, 'nuclear': 3, 'year': 3, 'presid': 2, 'make': 2, 'great': 2, 'oil': 2, 'young': 2, 'green': 2, 'abstain': 2, 'next': 2, 'citi': 2, 'opinion': 2, 'poll': 2, 'show': 2, 'far': 2, 'right': 2, 'april': 2, 'attract': 2, 'prime': 2, 'minist': 2, 'war': 2, 'centuri': 2, 'climat': 2, 'old': 2, 'build': 2, 'also': 2, 'cost': 2, 'michel': 1, 'rose': 1, 'reuter': 1, 'emmanuel': 1, 'promis': 1, 'stop': 1, 'use': 1, 'sourc': 1, 'pitch': 1, 'fear': 1, 'could': 1, 'week': 1, 'elect': 1, 'ralli': 1, 'mediterranean': 1, 'massiv': 1, 'firebrand': 1, 'jean': 1, 'luc': 1, 'sought': 1, 'widen': 1, 'small': 1, 'lead': 1, 'rival': 1, 'marin': 1, 'ahead': 1, '24': 1, 'presidenti': 1, 'race': 1, 'fought': 1, 'contend': 1, 'chose':

In [140]:
# !pip install collection

  Created wheel for collection: filename=collection-0.1.6-py3-none-any.whl size=5117 sha256=ff48d5ecdbfe7edaba67baa4038cbd6240b3af83b3a397de1d5bc1af07b15ce4
  Stored in directory: c:\users\bitcamp\appdata\local\pip\cache\wheels\b8\eb\72\804383af94abbb9ac1893ace85e5620780f0819144486a8bc1
Successfully built collection


In [ ]:
# CNBC 

In [88]:
cd C:/Users/bitcamp/Project4/best_model/

C:\Users\bitcamp\Project4\best_model


In [77]:
pwd

'C:\\Users\\bitcamp\\Project4\\data\\카테고리뉴스_final-20220423T080818Z-001\\카테고리뉴스_final'

In [89]:
import os
# csv파일 경로 리스팅 
data_lis = os.listdir('./')
data_lis

['best_mode5l.h5',
 'best_model1.h5',
 'best_model1.txt',
 'best_model2.h5',
 'best_model2.txt',
 'best_model3.h5',
 'tokenizer.pickle']

In [79]:
# csv파일 여러개 읽어와서 합치기 
df_cnbc = pd.DataFrame()

for files in data_lis:
    # advertisement 빼고 합치기 
    if "oil" in files:
        df = pd.read_csv(files,index_col=0)
        df_cnbc = pd.concat([df_cnbc,df])  

In [80]:
df = df_cnbc

In [85]:
df['content']=df['content'].apply(preprocessing)

In [118]:
preprocessed_sentences = df['content']

In [91]:
preprocessed_sentences

0     [oil, market, tighten, rbc, capit, market, thi...
1     [stock, futur, slightli, lower, monday, morn, ...
2     [presid, joe, biden, thursday, announc, histor...
3     [presid, joe, biden, top, econom, advisor, sug...
4     [oil, price, lower, thursday, white, hous, ann...
                            ...                        
45    [russia, ukrain, invas, could, set, motion, en...
46    [oil, price, edg, higher, settl, green, monday...
47    [ralli, kick, june, boost, oil, price, highest...
48    [oil, price, risen, worri, washington, possibl...
49    [oil, price, rose, two, week, monday, presid, ...
Name: content, Length: 699, dtype: object

In [102]:
# Investing.com 뭉태기뉴스


In [103]:
cd C:\Users\bitcamp\Project4\data\investing.com news-20220425T032341Z-001\investing.com news\commodities-news

C:\Users\bitcamp\Project4\data\investing.com news-20220425T032341Z-001\investing.com news\commodities-news


In [104]:
import os
# csv파일 경로 리스팅 
data_lis = os.listdir('./')
data_lis

['news_commodities-news_00001.csv',
 'news_commodities-news_00002.csv',
 'news_commodities-news_00003.csv',
 'news_commodities-news_00004.csv',
 'news_commodities-news_00005.csv',
 'news_commodities-news_00006.csv',
 'news_commodities-news_00007.csv',
 'news_commodities-news_00008.csv',
 'news_commodities-news_00009.csv',
 'news_commodities-news_00010.csv',
 'news_commodities-news_00011.csv',
 'news_commodities-news_00012.csv',
 'news_commodities-news_00013.csv',
 'news_commodities-news_00014.csv',
 'news_commodities-news_00015.csv',
 'news_commodities-news_00016.csv',
 'news_commodities-news_00017.csv',
 'news_commodities-news_00018.csv',
 'news_commodities-news_00019.csv',
 'news_commodities-news_00020.csv',
 'news_commodities-news_00021.csv',
 'news_commodities-news_00022.csv',
 'news_commodities-news_00023.csv']

In [105]:
# csv파일 여러개 읽어와서 합치기 
df_cnbc = pd.DataFrame()

for files in data_lis:
    # advertisement 빼고 합치기 
    if "com" in files:
        df = pd.read_csv(files,index_col=0)
        df_cnbc = pd.concat([df_cnbc,df])  

In [106]:
df = df_cnbc

In [110]:
df['content']=df['content'].dropna().reset_index().drop('index',axis=1)

In [114]:
df = df.reset_index().drop('index',axis=1)

In [117]:
df['content'] = df['content'].apply(preprocessing)

In [162]:
df['category'] = 리스
df

,company,date,title,content,category
0,By Reuters,"<span>3 hours ago (Apr 17, 2022 05:31AM ET)</s...",Tunisia says countries offer help to tackle im...,"[tarek, amara, tuni, reuter, countri, offer, h...",2
1,By Investing.com,"<span>5 hours ago (Apr 17, 2022 03:32AM ET)</s...",Energy & Precious Metals - Weekly Review and O...,"[barani, krishnan, invest, com, europ, forthco...",1
2,By Reuters,"<span>8 hours ago (Apr 17, 2022 12:41AM ET)</s...",Mexico's Pemex says fire under control at Sali...,"[mexico, citi, reuter, mexican, state, oil, co...",2
3,By Reuters,"<span>14 hours ago (Apr 16, 2022 06:05PM ET)</...",New bird flu cases identified in Pennsylvania ...,"[kanishka, singh, washington, reuter, new, bir...",4
4,By Oilprice.com,NaN,NaN,"[michel, rose, marseil, franc, reuter, presid,...",3
...,...,...,...,...,...
38368,By Reuters,"<span>Nov 22, 2010 03:04PM ET</span>","METALS-Copper slips on dollar, China demand wo...","[correct, lead, paragraph, say, mani, counterf...",4
38369,By Reuters,"<span>Nov 15, 2010 12:44AM ET</span>",GRAINS-Corn up 2 pct on talk of Chinese buying...,"[bartosz, dabrowski, reuter, lind, world, larg...",1
38370,By Reuters,"<span>Nov 09, 2010 11:16AM ET</span>",PRECIOUS-Gold hits new record as euro zone fea...,"[peter, nurs, invest, com, oil, price, weaken,...",0
38371,By Reuters,"<span>Nov 04, 2010 03:48AM ET</span>","GRAINS-Soy rises 1 pct, wheat firms after Fed ...","[alex, lawler, london, reuter, opec, said, thu...",0


In [163]:
# 1. investing.com 데이터 불러오기 
# 2. 데이터 불러와서 content 분류하기 
# 3. content데이터 전처리하기 
# 4. 로드한 pickle데이터로 패딩하기
# 5. 로드한 모델로 분류하기
# 6. 분류한 predict값을 대입해서 비교해보기
# 7. 데이터프레임에 라벨을 붙여넣기 

In [ ]:
# 여러데이터를 불러와서 합치기 
# 먼저 그 데이터가 있는쪽의 경로를 들어감
# 그경로의 데이터를 리스팅함 
# 반복문으로 모든 데이터를불러와서 데이터프레임을 만듬
# 예측을 할 컬럼을 뽑아 전처리를 진행
# 모델 돌리기
# 예측값 뽑아서 라벨붙이기
# pickle과 모델은 best_model에 있음

In [ ]:
# 비지도 학습에서 받아온 CNBC 데이터 카테고리 35개를 라벨붙여서 
# LSTM에 훈련시킴
# 그 모델과 토크나이저로 investing 적용 ㄱ 

# 데이터 프레임 만들기

In [164]:
pwd

'C:\\Users\\bitcamp\\Project4\\best_model'